<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-json-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis JSON, Search and Query, e Vector Search (soon)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

## Objetivos do Workshop

Este notebook irá fazer uma breve introdução ao **Hash** e o **JSON** (data types nativos) para o **Redis**.\
Feito isso, vamos entender as diferenças entre **buscas de texto** VS **buscas semânticas**, e o conceito de **Vector Databases**.

Entretanto, o foco deste workshop permanece no `JSON + Search and Query`, e como vocês podem utilizar o Redis como Banco de Dados de Documentos, primário, com toda a durabilidade e confiabilidade que esperam de um Banco de Dados Enterprise.
`

Espero que gostem! 🖖

## Setup Rápido - e testes pra ver se tá tudo redondo antes de iniciar o lab

In [ ]:
# Vamos instalar a lib do redis escolhida para o teste
!pip install -q redis

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

#### Configurando e testando a conexão com o seu Redis Cloud

Coloque o endpoint host, port, e as credenciais pertinentes ao seu setup.

Vou deixar o meu DB mesmo aqui, como referência.

In [ ]:
# Testando a redis-cli
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="redis-19581.c308.sa-east-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT=19581
REDIS_PASSWORD="nhtuquVSLbh2kUt2I86z5QwGu3KrcaYx"

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test a Redis connection
!redis-cli $REDIS_CONN PING

# DANGER ZONE - CASO QUEIRA DAR UM REFRESH GERAL
# este comando abaixo ficará comentado, pois ele deleta todos os dados da sua base do Redis Cloud
!redis-cli $REDIS_CONN FLUSHDB

In [ ]:
# Testando via Python (redis-py)
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
r.ping()

# Hashes e JSON - Dois velhos amigos

**No Redis, Hashes e JSON são tipos de dados nativamente suportados que oferecem estruturas eficientes para armazenar e manipular informações complexas.**

Um `Hash` no Redis é uma coleção de pares de campo-valor, semelhante a uma tabela de hash em outras linguagens de programação. Eu imagino ele como um mini redis, ou como um "JSON FLAT".\
Ideal para representar objetos com vários atributos, como um usuário com campos como nome, email e idade, Hashes são otimizados para operações rápidas de leitura e escrita, permitindo que cada campo dentro do hash seja acessado e modificado individualmente sem a necessidade de alterar toda a estrutura.

O `JSON` também é um data type nativo para o Redis. Nele, a utilização do tipo de dado JSON permite armazenar objetos completos e realizar consultas complexas diretamente na estrutura JSON armazenada. Isso é especialmente útil para aplicações que lidam com dados hierárquicos ou aninhados, como configurações de aplicativos, documentos ou dados de resposta de APIs, proporcionando uma maneira natural e flexível de gerenciar dados complexos dentro do Redis.\
Naturalmente, você pode manipular partes específicas do JSON, como vamos ver durante o Workshop.

**Importante:** Ambos data types suportam Indexação, habilitando eles para servirem de schema/model, e também para **Search and Query**.


## Redis Hash

No Redis, `Hashes` são coleções de pares de campo-valor, semelhantes a um `dicionário` em Python ou um `Map` em Java.

Eles são ideais para representar objetos com vários atributos, como um usuário com campos como nome, email e idade. Hashes são otimizados para operações rápidas de leitura e escrita, permitindo que cada campo dentro do hash seja acessado e modificado individualmente sem a necessidade de alterar toda a estrutura.

### Hands-on: Exercícios com Hashes no Redis

Vamos começar com alguns exemplos e comandos para praticar o uso de Hashes no Redis. Você pode executar esses comandos tanto no notebook quanto no Redis Insight.

#### Criar e manipular um Hash

**HSET:** Salva variáveis como campos em um hash no Redis.


```
# Execute no Redis Insights
HSET mvp:1 name "Ayrton" last_name "Senna" email "senninha@f1.io" age 34
```




In [ ]:
# Crie outro MVP utilizando o redis-cli por aqui mesmo:
!redis-cli $REDIS_CONN HSET mvp:2 name "Edson" last_name "Arantes do Nascimento" apelido "Pelé" email "pele@santosfc.com" age 82

#### Recuperar múltiplos valores de um hash

**HMGET:** Recupera múltiplos valores de um hash. Vamos pegar apenas alguns campos específicos, como nome, apelido e idade.

```
# Execute no Redis Insights
HMGET mvp:2 name apelido age
```

Vamos fazer o mesmo por aqui. Desta vez com o Python, nativamente.

In [ ]:
import redis

# Conectar ao Redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  encoding="utf-8",
  decode_responses=True
  )

# HMGET: Recuperar múltiplos valores
valores = r.hmget("mvp:2", "name", "apelido", "age")
print(valores)

#### Incrementar o valor de um campo numérico

Vamos usar o comando HINCRBY para incrementar o valor do campo age no hash.

**HINCRBY:** Incrementa o valor de um campo numérico. Um inteiro, neste caso.

Neste exemplo, Pelé já vai sair com 1000 gols.
Note que este campo `gol` nem existia antes no Hash `mvp:2`. Não é necessário um round-trip para definir este campo.

```
# Execute no Redis Insights, um por vez.
# Se quiser executar múltiplos, de uma vez, vá para a aba `Workbench` no Redis Insights.
HINCRBY mvp:2 gols 999
HINCRBY mvp:2 gols 2
HINCRBY mvp:2 gols -1

# E um get lazy mesmo, pra gente ver como está o nosso amigo Pelé:
HGETALL mvp:2
```

#### Deletar um campo de um hash

Como o Pelé ainda está fazendo gols no céu, acho difícil a gente tentar controlar isso aqui no Redis. Vamos resolver isso.

Vamos usar o comando `HDEL` para deletar o campo gols do hash `mvp:2`.

**HDEL:** Deleta um campo de um hash.


In [ ]:
import redis

# Conectar ao Redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True,
  )

# HDEL: Deletar o campo 'gols' do hash 'mvp:2'
r.hdel("mvp:2", "gols")

# Verificar todos os campos do hash 'mvp:2' para confirmar a deleção
pelezinho = r.hgetall("mvp:2")
print("Hash mvp:2 = ", pelezinho)

# Notem que o Python já está parseando a saida como um dict, e os bytes como string
print(type(pelezinho))
print(type(pelezinho['apelido']))


#### Deletar um hash completamente

Para deletar o hash `mvp:2` completamente, você pode usar o comando `DEL` via redis-cli, por exemplo.

**DEL:** Deleta a chave especificada e todos os seus campos.

Execute o comando abaixo no **redis-cli** para deletar o hash `mvp:2`:

In [ ]:
!redis-cli $REDIS_CONN DEL mvp:2

## RedisJSON

**RedisJSON** adiciona o tipo de dado **JSON** ao Redis para que você possa trabalhar com dados JSON **nativamente** no Redis, sem tratar o JSON inteiro como uma grande string e constantemente serializar/desserializar JSON no cliente.

Isso também significa que eu estou criando um **modelo/schema**. Os campos do JSON são tipados e possuem metacaracterísticas que vão ajudar muito vocês na hora de buscar dados no Redis. Buscas multi-facetadas, de vetor, de geolocalização, com filtros e transoformações que ocorrerão do lado do Redis, evitando multiplos round trips para o backend.

*Em outras palavras, você tira o que quer do Redis indo só uma vez até ele, inclusive filtrando exatamente o que você quer que passe pelo fio até sua aplicação.*


-----


Com uma biblioteca cliente como Python, você pode usar comandos como `redis.json().get()` e `redis.json().set()`, e no Redis CLI, JSON.GET, JSON.SET entre outros.

Eu, particularmente, prefiro usar as bibliotecas de Object Mapping, como a [REDISOM](https://redis.io/docs/latest/integrate/redisom-for-python/).

-----

Veja a lista completa de comandos do RedisJSON aqui: https://redis.io/commands/?group=json

Documentação Python: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisjson-commands


### Hands-on: Exercícios com Hashes no Redis

#### Criar um Objeto JSON

Vamos começar com um exemplo de um colégio em Curitiba e salvar esse objeto JSON no Redis.



In [ ]:
import redis
from redis.commands.json.path import Path

# Conectar ao Redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True)

# JSON.SET: Criar um objeto JSON para um colégio em Curitiba
colegio = {
    "name": "Colégio Positivo",
    "description": "Localizado em Curitiba, o Colégio Positivo é conhecido por seu currículo abrangente e inovador, com foco em tecnologia e excelência acadêmica.",
    "class": "particular",
    "type": ["moderno"],
    "address": {"city": "Curitiba", "street": "Rua Prof. Pedro Viriato Parigot de Souza"},
    "students": 1500,
    "location": "-49.275437,-25.428356",
    "status_log": ["novo", "em operação"],
    "teachers": [
        {
            "name": "Maria Oliveira",
            "subjects": ["Biologia", "Química"]
        },
        {
            "name": "João Lima",
            "subjects": ["Matemática", "Física"]
        }
    ],
    "tags": "excelência acadêmica, inovação, tecnologia"
}

# Salvar o objeto JSON no Redis com a chave 'colegio:42'
r.json().set("colegio:42", '$', colegio)

# Verificar se o objeto foi salvo corretamente
colegio_salvo = r.json().get("colegio:42")
print(colegio_salvo)


#### Read: Exemplos de Leitura de JSON no Redis

Neste bloco, vamos usar comandos via redis-cli, por aqui mesmo. Poderíamos estar usando o Python, a API, etc.

Vamos usar o objeto JSON do "Colégio Positivo" em Curitiba que criamos anteriormente. Execute os seguintes comandos no redis-cli para ler de diversas maneiras interessantes.


In [ ]:
# 1. Pegar o JSON inteiro
# Este comando retorna todo o objeto JSON armazenado na chave 'colegio:42'
!redis-cli $REDIS_CONN JSON.GET colegio:42

# 2. Pegar propriedades específicas
# Aqui, pegamos apenas as propriedades 'name' e 'address.city' do objeto JSON
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.name $.address.city

# 3. Pegar uma propriedade aninhada
# Este comando retorna o nome do primeiro professor na lista de professores
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.teachers[0].name

# 4. Pegar o primeiro elemento de um array
# Este comando retorna o primeiro status do log de status
!redis-cli $REDIS_CONN JSON.GET colegio:42 $.status_log[0]

#### Write: Exemplos de Escrita de JSON no Redis

Vamos mostrar como interagir com campos específicos do seu JSON. Olha que interessante (vamos de Python dessa vez):



In [ ]:
import redis
from redis.commands.json.path import Path

# Conectar ao Redis (usando variáveis de ambiente configuradas)
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD,
  decode_responses=True
)

# Imprimir o estado inicial do array status_log e do campo students
status_inicial = r.json().get("colegio:42", Path("$.status_log"))
alunos_iniciais = r.json().get("colegio:42", Path("$.students"))
print("Status inicial:", status_inicial)
print("Número de alunos inicial:", alunos_iniciais)

print("----------------------------------------------------------------")

# 6. Adicionar um novo status ao array status_log
r.json().arrappend("colegio:42", Path("$.status_log"), "em expansão")
status_atualizado = r.json().get("colegio:42", Path("$.status_log"))
print("Status atualizado:", status_atualizado)

# 7. Incrementar o número de alunos em 10
r.json().numincrby("colegio:42", Path("$.students"), 10)
alunos_atualizados = r.json().get("colegio:42", Path("$.students"))
print("Número de alunos atualizado:", alunos_atualizados)

# Search and Query: vamos começar a navegar pelo mundo das consultas avançadas

Vamos iniciar carregando alguns outros Colégios conhecidos por nós, Brasileiros.

Basta executar o próximo bloco, e vamos em frente.

In [ ]:
schools = [
    {
        "name": "Colégio Bandeirantes",
        "description": "Abrangendo 10 estados, o currículo premiado desta escola inclui um sistema de leitura abrangente (do reconhecimento de letras e fonética à leitura de livros completos), além de matemática, ciências, estudos sociais e até filosofia.",
        "class": "independente",
        "type": ["tradicional"],
        "address": {"city": "São Paulo", "street": "Rua Estela"},
        "students": 342,
        "location": "-46.633308,-23.550520",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Ana Silva",
                "subjects": ["Matemática", "Física"]
            },
            {
                "name": "Carlos Souza",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "excelência acadêmica, tecnologia, inovação, java"
    },
    {
        "name": "Escola Maria Imaculada",
        "description": "A Garden School é uma nova e inovadora experiência de ensino e aprendizado ao ar livre, oferecendo atividades ricas e variadas em um ambiente natural para crianças e famílias.",
        "class": "estadual",
        "type": ["floresta", "montessori"],
        "address": {"city": "São Paulo", "street": "Avenida Vicente Rao"},
        "students": 1452,
        "location": "-46.699687,-23.621993",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Mariana Costa",
                "subjects": ["Biologia", "Química"]
            },
            {
                "name": "Rafael Lima",
                "subjects": ["Português", "Literatura"]
            }
        ],
        "tags": "educação ao ar livre, inovação, sustentabilidade"
    },
    {
        "name": "Colégio São Luís",
        "description": "A Gillford School é um centro de aprendizado inclusivo que acolhe pessoas de todos os estilos de vida, convidando-as a assumir seu papel como agentes regenerativos, criando novos caminhos para o futuro e incitando um movimento internacional de transformação cultural, territorial e social.",
        "class": "privada",
        "type": ["democrática", "waldorf"],
        "address": {"city": "Rio de Janeiro", "street": "Rua Haddock Lobo"},
        "students": 721,
        "location": "-46.660213,-23.558704",
        "status_log": ["novo", "em operação", "fechada"],
        "teachers": [
            {
                "name": "Beatriz Mendes",
                "subjects": ["Filosofia", "Sociologia"]
            },
            {
                "name": "Fernando Almeida",
                "subjects": ["Artes", "Educação Física"]
            }
        ],
        "tags": "inclusão, transformação social, interdisciplinaridade"
    },
    {
        "name": "Escola Sesc de Ensino",
        "description": "A filosofia por trás da Forest School baseia-se no desejo de proporcionar às crianças pequenas uma educação que incentive a apreciação do mundo amplo na natureza, enquanto alcançam independência, confiança e alta autoestima.",
        "class": "independente",
        "type": ["floresta", "montessori", "democrática"],
        "address": {"city": "Rio de Janeiro", "street": "Rua Jacarepaguá"},
        "students": 1000,
        "location": "-43.375162,-22.972250",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Luciana Oliveira",
                "subjects": ["Ciências", "Matemática"]
            },
            {
                "name": "Paulo Santos",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "natureza, independência, confiança"
    },
    {
        "name": "Colégio Móbile - Perfeito",
        "description": "A Escola Móbile oferece um programa acadêmico rigoroso combinado com foco em pensamento crítico e criatividade, preparando os alunos para o ensino superior e além.",
        "class": "privada",
        "type": ["tradicional"],
        "address": {"city": "Curitiba", "street": "Rua Cotovia"},
        "students": 600,
        "location": "-46.676201,-23.592920",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Juliana Pereira",
                "subjects": ["Química", "Biologia"]
            },
            {
                "name": "Ricardo Fernandes",
                "subjects": ["Português", "Inglês"]
            }
        ],
        "tags": "rigor acadêmico, pensamento crítico, criatividade"
    },
    {
        "name": "Colégio Santa Cruz",
        "description": "O Colégio Santa Cruz é conhecido por seu forte desempenho acadêmico e compromisso com a responsabilidade social, promovendo um senso de comunidade entre os alunos.",
        "class": "privada",
        "type": ["tradicional", "religiosa"],
        "address": {"city": "Belo Horizonte", "street": "Rua Orobó"},
        "students": 300,
        "location": "-46.712750,-23.547680",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Gabriela Rocha",
                "subjects": ["Religião", "Filosofia"]
            },
            {
                "name": "Eduardo Matos",
                "subjects": ["Educação Física", "Ciências"]
            }
        ],
        "tags": "desempenho acadêmico, responsabilidade social, comunidade"
    },
    {
        "name": "Colégio Rio Branco",
        "description": "O Colégio Rio Branco oferece um currículo diversificado com forte ênfase em línguas, artes e ciências, preparando os alunos para a cidadania global.",
        "class": "privada",
        "type": ["internacional"],
        "address": {"city": "São Paulo", "street": "Rua Alves Guimarães"},
        "students": 50,
        "location": "-46.670912,-23.555451",
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Marcelo Andrade",
                "subjects": ["Inglês", "Espanhol"]
            },
            {
                "name": "Sofia Ribeiro",
                "subjects": ["Artes", "História"]
            }
        ],
        "tags": "línguas, artes, cidadania global, java"
    }
]



# carregar os dados do Redis... algumas escolas do Brasilzão
for id, school in enumerate(schools):
    #print(school)
    r.json().set(f"school_json:{id}", '.', school)

Execute estes dois comandos abaixo, para alguns exemplos e testes simples:

In [ ]:
# Pegar um JSON inteiro, via JSON.GET, da raiz $
!redis-cli $REDIS_CONN JSON.GET school_json:1 $
# Listar as keys do schema school_json que criamos juntos no python acima
!redis-cli $REDIS_CONN keys 'school_json:*'

Agora algo equivalente, mas com Python! Bem simples, né?

In [ ]:
# Agora com Python!
# Pegue um JSON direto, desde a raiz com $
res=r.json().get('school_json:0','$')
print(res)


Seguindo com Python, mas com alguns exemplos mais legais

In [ ]:
# Retorne apenas a propriedade nome
res=r.json().get('school_json:0','$.name')
print(res)

# Embedded object ($.address) - Um Address pertence a uma Escola - 1:1
res=r.json().get('school_json:0','$.address')
print(res)

# E agora um elemento de um array - o primeiro elemento
res=r.json().get('school_json:0','$.status_log[0]')
print(res)

In [ ]:
# Ler a quantidade de students de uma escola X. Neste caso, a primeira!
students=r.json().get('school_json:0','$.students')
print(students)

# Mudar pra 350 alunos e refazer o mesmo comando.
# sempre atomico, sempre O(1).
r.json().set('school_json:0','$.students',350)
r.json().get('school_json:0','$.students')

In [ ]:
# Incremento atomico do numero de estudandes da escola-alvo... sempre O(1)
# Pode clicar algumas vezes! O numincrby vai retornar o novo numero de Estudantes da Escola, assim como na CLI/API.
r.json().numincrby('school_json:0','$.students',1)

## RediSearch - Dando mais poderes para o que já é brabo

**RediSearch** adiciona a capacidade de consultar dados nas suas estruturas de dados **HASH** ou **JSON**, **essencialmente transformando o Redis em um banco de dados de documentos.** Sim.

Com o RediSearch, você declara índices uma vez e, em seguida, cada objeto de banco de dados que corresponder ao prefixo definido no índice será automaticamente e em tempo real adicionado ao índice.

Naturalmente, o que já estiver no Redis será indexado também, a não ser que você não queira. Existem vários designs diferentes para esse tema, incluindo indices pequenos de com curto lifespan. Alguns clientes usam esse modelo para tornar a experiencia de um cliente mais fluida e com menor percepção de latência.\
*Por exemplo, você cria um determinado indice que só vai servir durante a jornada de um cliente logado, até o checkout, e depois vc manda o índice pro beleléu.*


-----

Para a lista completa de comandos do RediSearch, veja: https://redis.io/commands/?group=search

Documentação Python: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisearch-commands

### Configurando o Índice no Redis (com tudo, incluindo Geolocation)

Vamos criar um **Index** no Redis. Vamos mostrar como configurar o Redis para tratar JSONs como objetos, considerando seus diferentes atributos e propriedades.

Aqui, além de configurar campos de texto, tags e numéricos, vamos também configurar o campo de geolocalização.

Vamos começar excluindo qualquer índice existente para evitar conflitos. Em seguida, criaremos um novo índice com um esquema completinho.

---
**Disclaimers:**

Sobre o Geolocation, que veremos apenas lá no fim: Este campo permitirá consultas baseadas em coordenadas geográficas, como buscar escolas dentro de um determinado raio a partir de um ponto específico.

*Este exemplo não inclui campos de vetor (Redis como um VectorDB) ainda, para manter o foco. Vamos explorar vetores em exercícios futuros.*

---

Então, vamos lá!

In [ ]:
# Aqui eu quero trazer a sua atencao para o schema.
# Notem como eu explico para o Redis como tratar o JSON como um objeto mesmo, considerando seus atributos/propriedades.
# estamos apenas comecando! esse indice nem vetor (ainda, rsrs, vamos ver juntos em breve)
# PORÉM, AQUI, eu já vou parsear o campo de GEOLOCATION, como uma tupla de (longitude,latitude)

from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
    GeoField,

)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

# Excluir o índice se ele já existir
try:
    r.ft("idx:schools_json").dropindex(delete_documents=False)
except:
    pass

# E aqui criamos o index pra valer. Nem vou colocar vector e geoloc agora, pra gente nao confundir.
schema = (
    TextField("$.name", as_name="name"),
    TextField("$.description", as_name="description"),
    TagField("$.address.city", as_name="city"),
    NumericField("$.students", as_name="students"),
    TagField("$.tags", as_name="tags", separator=","),
    GeoField("$.location", as_name="location")
    )
r.ft("idx:schools_json").create_index(schema,
                    definition=IndexDefinition(prefix=["school_json:"],
                    index_type=IndexType.JSON)
                    )

### Testando o Setup com uma Full-Text search

Não quero te confundir agora, mas os casos de uso vão do básico até avançadíssimos, como você pode ver [aqui](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/).

No exemplo a seguir, vamos realizar uma Full-Text Search buscando pelo termo `rigoroso`.

Como o campo `Description` do nosso schema `School` está sendo indexado como **TextField**, vamos enctontrar o Colégio que possui esse termo lá.


In [ ]:
import pandas as pd
# Retorne o Documento inteiro! Aqui fizemos uma Full-Text Search por rigoroso.
res=r.ft("idx:schools_json").search("rigoroso")
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

In [ ]:
# Retorne apenas alguns campos, para evitar passar coisas e metadados desnecessários pelo fio, do Redis para sua App.
query=Query("rigoroso") \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name")
res=r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

Vamos fazer uma busca um pouco mais interessante.

Quero encontrar escolas em São Paulo (TAG), que possua de 300 a 2000 alunos.
Neste exemplo, a TAG foi um exact match, atômico.

Por conta deste campo, eu tenho um alias que ignora os aninhamentos, onde a cidade (City) pertence ao endereço (Address) da entidade-pai (School).

```
# Durante a criação do Index, falamos pro Redis que Cidade é uma TAG.
<...>
TagField("$.address.city", as_name="city")
<...>
```

In [ ]:
# Busca Multi-Field, um pouquinho mais complexa
# Notem o uso de {} para exact match de tags
# Notem tambem que estamos vendo que o Redis eh binary-safe...
# A Cidade, sendo uma tag, casa certinho, considerando o 'ã' de São Paulo
query=Query('@city:{São Paulo} @students:[300, 2000]') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res=r.ft("idx:schools_json").search(query)
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df


# Casos de uso poderosos e mais avançados
(não são complexos, é tranquilo entender)

## Busca por Tags - Encontrando escolas com características específicas

Nesta seção, vamos realizar buscas utilizando tags.

Tags permitem buscas exatas em campos específicos. Mas isso não quer dizer que você não pode usar wildcards.

In [ ]:
# Busca por tags (mesmo que tenham wildcard) - binary safe - usando o melhor idioma do PLANETA: PT-BR
query = Query('@tags:{inovaçã*}') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print(res_df)

## Busca por Intervalo - Quantos alunos estão matriculados?

Busca por intervalo é o que você vai encontrar na doc oficial como "Range queries"

Para realizar buscas por intervalos em campos numéricos, como a quantidade de alunos em uma escola, podemos utilizar essas consultas de intervalo.

**Não se assustem com `inf+` e `inf-`!**\
Esses termos são utilizados para representar o intervalo máximo e mínimo, respectivamente. inf+ significa infinito positivo (o valor mais alto possível) e inf- significa infinito negativo (o valor mais baixo possível). Eles são úteis quando você quer incluir todos os valores acima ou abaixo de um determinado limite.

In [ ]:
# Aqui, vamos realizar buscas baseadas em intervalos numéricos. Neste caso, a quantidade de alunos.
# TODO - Colocar mais ranges e mudar as cidades

# Busca por escolas com entre 300 e 1000 alunos (inclusivo)
search = '@students:[300 1000]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm entre 300 e 1000 estudantes (inclusive)")
print("Search: {0}".format(search))
print("")
print(res_df)
print("--------------------------------------------------------------------")

# Busca por escolas com mais de 1000 alunos (exclusivo)
# Note que o Escola Sesc de Ensino não vai aparecer, pois tem exatos 1000 alunos
search = '@students:[(1000 +inf]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm mais de 1000 estudantes (exclusive)")
print("Search: {0}".format(search))
print("Comment: Note que a Escola Sesc de Ensino não vai aparecer, pois tem exatos 1000 alunos")
print("")
print(res_df)
print("--------------------------------------------------------------------")


# Busca por escolas com menos de 300 alunos (inclusivo)
search = '@students:[-inf 300]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm menos de 300 estudantes (inclusive)")
print("Search: {0}".format(search))
print("")
print(res_df)
print("--------------------------------------------------------------------")


# Busca por escolas com menos de 300 alunos (exclusivo)
search = '@students:[0 (300]'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm menos de 300 estudantes (exclusive)")
print("Search: {0}".format(search))
print("Comment: Note que o Colégio Santa Cruz não vai aparecer, pois tem exatos 300 alunos")
print("")
print(res_df)
print("--------------------------------------------------------------------")



## Fuzzy Match - "Serasse" eu sei escrever essa palavra?

Pra fechar, apenas um exemplo leve de comunicação realista: podemos estar buscando sem saber escrever direito o termo. Esse é apenas um das dezenas de exemplos que vocês vão ter acesso na doc do Redis. Tem de redução de radicais, levando em consideração o contexto e cultura de uma determinada Língua, etc.

Aqui, vamos apenas fazer uma full-text search procurando por alguma coisa meio esquisita, e vamos ver se o Redis consegue ajudar.

In [ ]:
# Fuzzy matching!
# %algumacoisa% significa todos os termos com distância de Levenshtein de 1 em relação a ele.
# Use múltiplos pares de % para aumentar a distância de Levenshtein.

query=Query('%%Bondeirants%%') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res=r.ft("idx:schools_json").search(query)
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

### Suporte a Sinônimos

O Redis Stack suporta a busca por palavras sinônimas definidas pela estrutura de dados de sinônimos. Isso permite que, ao buscar por uma palavra, documentos contendo suas sinônimas também sejam retornados.

A estrutura de dados de sinônimos é um conjunto de grupos, cada um contendo termos sinônimos. Por exemplo, o seguinte conjunto de sinônimos contém três grupos, e cada grupo contém três termos sinônimos:

- {menino, criança, bebê}
- {menina, criança, bebê, girl}
- {mulher, pessoa, adulto}

**Pensando nestes três `Sets`, podemos afirmar que:**

Quando esses grupos estão na estrutura de dados de sinônimos, é possível buscar por "criança" e receber documentos contendo "menino", "menina", "girl", "criança" e "bebê".

In [ ]:
# Busca com Sinônimos - Procurando por termos relacionados entre si
# Vamos utilizar um set aqui, bem simples.
# E vamos usar um exemplo nada a ver, justamente para reforçar que nenhum modelo de MachineLearning precisou ser usado para aproximar os termos.
# Essa parte mais doida, vamos ver nos exercícios de Vector Search (vector similarity, cosine, etc)

# Configurando grupo de sinônimos com termos fora do contexto escolar
# Aqui é apenas uma brincadeira, com coisas bem desconexas mesmo
r.ft("idx:schools_json").synupdate("synonym_maluco_1", False, "Perfeito", "CristianoRonaldo", "CR7")

# Finalmente, a busca utilizando este sinônimo curioso
search = 'CristianoRonaldo'
query = Query(search) \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.description", as_field="description")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Busca por 'Cristiano Ronaldo' usando sinônimos (inclui 'Perfeito' e 'CR7')")
print("Search: {0}".format(search))
print(res_df)
print("--------------------------------------------------------------------")


## Suporte a Stemming - uma forma bonita de dizer que vamos buscar pelo radical da palavra

O **Redis Stack** suporta **Stemming**, permitindo a adição da forma base de uma palavra ao índice (radical). Isso possibilita que uma busca por "contratando" também retorne resultados para "contratar" e "contratado".

O suporte atual a Stemming é baseado na biblioteca **Snowball Stemmer**, que suporta a maioria das línguas europeias, bem como árabe e outras.

Para definir qual idioma o Stemmer deve aplicar ao construir o índice, você precisa especificar o parâmetro LANGUAGE ao criar o índice ou para um campo específico.

**Neste caso, vou usar um exemplo comum do Inglês.**

Vamos buscar pelo termo `hiring`.\
Vamos ver o que ele traz de stemming, considerando o verbo "to `hire`".



In [ ]:
### Suporte a Stemming

# Criação do índice com suporte a Stemming em inglês
# Para criar um índice com suporte a stemming, definimos os campos TextField apropriados.
from redis.commands.search.field import TextField, TagField, NumericField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
import pandas as pd

# Agora, pessoal, vou criar um outro índice. Mas é apenas pra gente não confundir as paradas.
# Na verdade, vocês podem criar e destruir índices ao longo da operação em Prod... isso é normal
# Temos clientes que criam índices de curta-vida, durando apenas enquanto um determinado usuário está logado e interagindo com o sistema.
# Excluir o índice se ele já existir
try:
    r.ft("idx:schools_stemming").dropindex(delete_documents=False)
except:
    pass

# Criação do índice com suporte a stemming
schema = (
    TextField("$.name", as_name="name"),
    TextField("$.description", as_name="description")
)

# Usando o conceito de Index Definition dessa lib do Python, pra injetar e explicar pro Redis que falamos inglês por aqui
r.ft("idx:schools_stemming").create_index(schema, definition=IndexDefinition(prefix=["school_json:"], index_type=IndexType.JSON, language='english'))

# Adicionar documentos com variações da palavra "hire"
r.json().set('school_json:10', '$', {"name": "School Hiring", "description": "just to avoid confusion"})
r.json().set('school_json:11', '$', {"name": "College Hire", "description": "just to avoid confusion"})
r.json().set('school_json:12', '$', {"name": "Academy of Hired", "description": "just to avoid confusion"})

# Listar todos os documentos no índice
print("--------------------------------------------------------------------")
print("Listando todos os documentos no índice")
print("--------------------------------------------------------------------")
search = '*'
query = Query(search).return_field("$.name", as_field="name")
res = r.ft("idx:schools_stemming").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print(res_df)

# Busca com Stemming para "hiring"
search = '@name:(hiring)'

query_params = {
  "LANGUAGE": "english"  # Gabs: parece estar sendo ignorado - abrir uma issue rapidola
}

query = Query(search) \
    .return_field("$.name", as_field="name") \
    .return_field("$.description", as_field="description")

res = r.ft("idx:schools_stemming").search(query, query_params)

res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Busca por 'hiring' com suporte a Stemming")
print("Search: {0}".format(search))
print(res_df)
print("--------------------------------------------------------------------")


## Suporte a Correspondência Fonética

A correspondência fonética permite que você encontre termos baseados na sua pronúncia. Isso é particularmente útil ao buscar nomes de pessoas, onde variações na escrita podem ocorrer, mas a pronúncia permanece semelhante.

O **Redis Stack** suporta a correspondência fonética usando o algoritmo **Double Metaphone (DM)**, que é eficaz para várias línguas latinas, incluindo inglês, francês, português e espanhol. Ao definir um campo de texto com a opção PHONETIC, os termos nesse campo serão indexados tanto pelo seu valor textual quanto pela sua representação fonética.

Vamos parar de usar Python agora, pra fechar com chave de ouro.\
Podem seguir fazendo por aqui, visto que temos um `redis-cli` disponível.

Entretanto, minha recomendação é fazer esse último aqui direto no **Redis Insights (GUI)**

Vamos lá!\
*Ah, e parabéns por ter chegado até aqui. Gostei muito de preparar esse material pra vocês.*


### Configuração do Índice com Correspondência Fonética

Comando para criar o índice:

In [ ]:
!redis-cli $REDIS_CONN FT.DROPINDEX idx:schools_phonetic || true
!redis-cli $REDIS_CONN FT.CREATE idx:schools_phonetic ON JSON PREFIX 1 school_json: LANGUAGE portuguese SCHEMA $.name AS name TEXT PHONETIC dm:pt $.description AS description TEXT

### Adicionar Documentos

Comandos para adicionar documentos:

In [ ]:
!redis-cli $REDIS_CONN JSON.SET school_json:30 $ "{\"name\": \"Escola João\", \"description\": \"Um ótimo lugar para aprender\"}"
!redis-cli $REDIS_CONN JSON.SET school_json:31 $ "{\"name\": \"Escola Jão\", \"description\": \"Uma excelente instituição educacional\"}"
!redis-cli $REDIS_CONN JSON.SET school_json:32 $ "{\"name\": \"Escola Joãozinho\", \"description\": \"Educação de primeira qualidade\"}"

### Executar Consultas Fonéticas

Comandos para executar consultas fonéticas, e com `SORT DESC BY name` (só pra ficar mais legal):

In [ ]:
!echo "##############################################################################################################"
# Buscar por "João" que deve retornar documentos com "João" e "Jão", em ordem decrescente
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "João" LANGUAGE portuguese SORTBY name DESC RETURN 1 name
!echo "##############################################################################################################"
!echo ""
!echo "##############################################################################################################"
# Buscar por "Jão" que deve retornar documentos com "Jão" e "João" também, mas em ordem crescente agora
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "Jão" LANGUAGE portuguese SORTBY name ASC RETURN 1 name
!echo "##############################################################################################################"
!echo ""
!echo "##############################################################################################################"
# Buscar por "Joãozinho" que deve retornar documentos com "Joãozinho". Neste caso, só tem um.
!redis-cli $REDIS_CONN FT.SEARCH idx:schools_phonetic "Joãozinho" LANGUAGE portuguese SORTBY name DESC
!echo "##############################################################################################################"


# Busca Complexa no Redis: O Grand Finale

Nesta última parte do nosso workshop, vamos utilizar o Redis para realizar uma busca complexa combinando várias funcionalidades avançadas de busca que aprendemos. Vamos explorar:

    Fuzzy Search: Para encontrar termos que são similares, mas não exatamente iguais, ao termo de busca.
    Full Text Search: Para buscar termos em campos de texto.
    Tag Search: Para encontrar correspondências exatas em campos de tags.
    Geolocation Search: Para buscar documentos dentro de um determinado raio de uma localização específica.
    Range Search: Para buscar documentos com valores numéricos dentro de um intervalo específico.
    Sorting: Para ordenar os resultados por um campo numérico.
    # e, no próximo notebook... vamos partir pro Vector e entrar no mundo de LLM e Machine Learning.

Nosso objetivo será encontrar escolas com base em uma combinação desses critérios e retornar apenas o nome e a quantidade de alunos, ordenados pelo número de alunos (do maior pro menor).

### Olha que legal:

In [ ]:
# Busca complexa
search = '%%Colejio%% (@name:Colégio|Banana) @city:{São Paulo} @tags:{tecnologia|ambiental|java} @location:[-46.633308 -23.550520 100000 km] @students:[25 400]'
query = Query(search) \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students") \
    .sort_by("students", asc=False)
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])

print("--------------------------------------------------------------------")
print("Busca Complexa: Fuzzy, Full Text, Tag, Geoloc, Range e Sort")
print("Search: {0}".format(search))
print(res_df)
print("--------------------------------------------------------------------")

# Aggregations - como usar o FT.AGGREGATE

As agregações são uma maneira de processar os resultados de uma consulta de pesquisa, agrupar, ordenar e transformá-los - e extrair insights analíticos deles. Assim como consultas de agregação em outros bancos de dados e motores de busca, elas podem ser usadas para criar relatórios analíticos ou realizar consultas no estilo de Pesquisa Facetada.

Por exemplo, podemos agrupar escolas por cidade e contar escolas por grupo, nos dando o número de escolas por cidade. Ou poderíamos agrupar por classe escolar (particular/estatal) e ver o número médio de alunos por grupo.





In [ ]:
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search import reducers

# helper function para mostrar os resultados do redis.ft().search() como um dataframe, pra quem curte pandas lib
def display_ft(res):
  if res.total==0:
    print("No matches found")
  else:
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    display(res_df)

# helper function para traduzir o resultado do FT.AGGREGATE para um dataframe e entao mostrar o resultado
def display_ft_agg(res):
  data = res.rows
  data = [[item for item in sublist] for sublist in data]
  column_dict = {}
  for sublist in data:
      for i in range(0, len(sublist), 2):
          column_name = sublist[i]
          column_value = sublist[i + 1]
          column_dict.setdefault(column_name, []).append(column_value)
  df = pd.DataFrame(column_dict)
  display(df)


# Execute uma agregacao por cidade, e conte o numero de escolas por cidade (parecido com um SQL da vida).
request = AggregateRequest(f'*').group_by('@city', reducers.count().alias('count'))
res = r.ft("idx:schools_json").aggregate(request)
#print(res.rows)
display_ft_agg(res)


# Execute uma agregacao por cidade, e conte o numero de estudantes por cidade
# Note que agora estamos mandando um SUM, apra somar os students, e nao mais um COUNT
request = AggregateRequest(f'*').group_by('@city', reducers.sum('@students').alias('students_count'))
res = r.ft("idx:schools_json").aggregate(request)
display_ft_agg(res)


## Bonus!!! Verificação Ortográfica com Redis e FT.SEARCH (doideira)

O comando `FT.SPELLCHECK` realiza a correção ortográfica em uma consulta, retornando sugestões para termos incorretos. Podemos especificar a distância máxima de **Levenshtein** para sugestões de ortografia e usar **dicionários personalizados** para incluir ou excluir termos.

Eu ainda não montei um dicionário personalizado, mas vou assim que tiver tempo.\
Quero fazer algo com regionalismo e dialetos.

Enfim, foco aqui!

Vamos criar um índice simples e adicionar alguns documentos para simular a correção ortográfica. Em seguida, usaremos o comando `FT.SPELLCHECK` para verificar a ortografia da palavra "Redis".

### Passo 1: Criação do Índice

Aqui vale lembrar que não precisamos ter vários índices. Eu estou criando mais só pro exercício ficar mais "atômico", rsrs.

In [ ]:
!redis-cli $REDIS_CONN FT.DROPINDEX idx:spellcheck || true
!redis-cli $REDIS_CONN FT.CREATE idx:spellcheck ON JSON PREFIX 1 doc: SCHEMA $.content AS content TEXT

### Passo 2: Adição de Documentos


In [ ]:
!redis-cli $REDIS_CONN JSON.SET doc:1 $ "{\"content\": \"Redis is an in-memory data structure store.\"}"
!redis-cli $REDIS_CONN JSON.SET doc:2 $ "{\"content\": \"Redis provides high availability through Redis Sentinel.\"}"
!redis-cli $REDIS_CONN JSON.SET doc:3 $ "{\"content\": \"Redis is used for caching, real-time analytics, and message brokering.\"}"

### Passo 3: Execução do Comando FT.SPELLCHECK

In [ ]:
# Verificação ortográfica para a palavra incorreta "Redsi"
!echo "##############################################################################################################"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Redsi DISTANCE 1
!echo "##############################################################################################################"

!echo ""

# Verificação ortográfica para a palavra incorreta "Rédiz" - com Levenshtein DISTANCE de 1
!echo "##############################################################################################################"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Rédiz DISTANCE 1
!echo "##############################################################################################################"

!echo ""

# Verificação ortográfica para a palavra incorreta "Rédiz" - com Levenshtein DISTANCE de 2 - agora vai rsrs
!echo "##############################################################################################################"
!redis-cli $REDIS_CONN FT.SPELLCHECK idx:spellcheck Rédiz DISTANCE 2
!echo "##############################################################################################################"